In [1]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from numba import jit

### Gray-flip function

In [2]:
@jit
def gray_flip(lista):

    N = len(lista)
    tau_list = lista
    k = tau_list[0]
    spins = [-1 for _ in range(N)]
    configs = []
    configs.append(spins.copy())

    while k < N:
        
        spins[k - 1] *= -1
        configs.append(spins.copy())
        
        try:
            tau_list[k - 1] = tau_list[k]
            tau_list[k] = k + 1
        except:
            print(k)
            break
        k = tau_list[0]
        

        if k != 1:
            tau_list[0] = 1
        
    return k, tau_list, configs


In [28]:
config_red2x2 = gray_flip([1, 2, 3, 4, 5])[2]
for i in config_red2x2:
    print(i)


[-1, -1, -1, -1, -1]
[1, -1, -1, -1, -1]
[1, 1, -1, -1, -1]
[-1, 1, -1, -1, -1]
[-1, 1, 1, -1, -1]
[1, 1, 1, -1, -1]
[1, -1, 1, -1, -1]
[-1, -1, 1, -1, -1]
[-1, -1, 1, 1, -1]
[1, -1, 1, 1, -1]
[1, 1, 1, 1, -1]
[-1, 1, 1, 1, -1]
[-1, 1, -1, 1, -1]
[1, 1, -1, 1, -1]
[1, -1, -1, 1, -1]
[-1, -1, -1, 1, -1]


### Energy-ising

In [3]:
@jit
def Nbr(lista, n, m):
    matrix = []
    dim = int(np.sqrt(len(lista)))
    x = 0
    while x < len(lista):
        matrix.append(lista[x:x+dim])
        x += dim

    spin = matrix[n][m]
    nbr_down = matrix[(n + 1) % dim][m]
    nbr_right = matrix[n][(m + 1) % dim]

    return spin, nbr_down, nbr_right
    

@jit
def energy_ising(configuration):
    dim = int(np.sqrt(len(configuration)))
    E = 0
    for i in range(dim):
        for j in range(dim):
            
            nbrs = Nbr(configuration, i, j)
            actual_spin = nbrs[0]
            spin_down = nbrs[1]
            spin_right = nbrs[2]

            E -= actual_spin * spin_down
            E -= actual_spin * spin_right
    
    return E

### Enumerate-ising

In [10]:
@jit
def enumerate_ising(N):
    densities = [0 for _ in range(0, 2*N+4, 4)]
    tau_list = [i for i in range(1, N+2)]
    configs = gray_flip(tau_list)[2]
    energy_list = [int(i) for i in range(0, 2*N + 4, 4)]
    for i in configs:
        E = int(energy_ising(i))
        if E in energy_list:
            index = energy_list.index(E)
            densities[index] += 1 

    return energy_list, densities

#### 2x2

In [5]:
energy2x2, dens2x2 = enumerate_ising(4)
for i in range(len(energy2x2)):
    print(f"E={energy2x2[i]}", f"N={dens2x2[i]}")


c:\Users\vicex\AppData\Local\Programs\Python\Python39\lib\site-packages\awkward\_cpu_kernels.py:9: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


E=0 N=12
E=4 N=0
E=8 N=4


#### 4x4

In [11]:
energy4x4, dens4x4 = enumerate_ising(16)
for i in range(len(energy4x4)):
    print(f"E={energy4x4[i]}", f"N={dens4x4[i]}")
print(dens4x4)

E=0 N=20524
E=4 N=13568
E=8 N=6688
E=12 N=1728
E=16 N=424
E=20 N=64
E=24 N=32
E=28 N=0
E=32 N=2
[20524, 13568, 6688, 1728, 424, 64, 32, 0, 2]
